# Flower Classification ML Model

This notebook contains the complete workflow for training and evaluating a flower classification model.

## Table of Contents
1. [Data Exploration](#data-exploration)
2. [Data Preprocessing](#data-preprocessing)
3. [Model Architecture](#model-architecture)
4. [Training](#training)
5. [Evaluation](#evaluation)
6. [Visualization](#visualization)
7. [Model Export](#model-export)

In [ ]:
# Import required libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Add src to path for importing our modules
sys.path.append('../src')
from preprocessing import DataPreprocessor
from model import ModelManager
from prediction import FlowerPredictor

print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

## 1. Data Exploration

In [ ]:
# Initialize data preprocessor
preprocessor = DataPreprocessor(data_dir='../data')

# Get dataset statistics
stats = preprocessor.get_dataset_statistics()
print("Dataset Statistics:")
print(f"Total images: {stats['total_images']}")
print(f"Training images: {stats['train_images']}")
print(f"Test images: {stats['test_images']}")
print(f"Dataset health: {stats['dataset_health']}")
print("\nClass distribution:")
for class_name, counts in stats['class_distribution'].items():
    print(f"  {class_name}: {counts['total']} total ({counts['train']} train, {counts['test']} test)")

In [ ]:
# Visualize class distribution
class_names = list(stats['class_distribution'].keys())
class_counts = [stats['class_distribution'][name]['total'] for name in class_names]

plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.bar(class_names, class_counts, color=['#ff7f7f', '#7f7fff', '#ffff7f'])
plt.title('Class Distribution (Total Images)')
plt.ylabel('Number of Images')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
plt.pie(class_counts, labels=class_names, autopct='%1.1f%%', colors=['#ff7f7f', '#7f7fff', '#ffff7f'])
plt.title('Class Distribution (Percentage)')

plt.tight_layout()
plt.show()

In [ ]:
# Display sample images from each class
def show_sample_images(data_dir, class_names, samples_per_class=3):
    fig, axes = plt.subplots(len(class_names), samples_per_class, figsize=(15, 12))
    
    for i, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, 'train', class_name)
        images = [f for f in os.listdir(class_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        for j in range(min(samples_per_class, len(images))):
            img_path = os.path.join(class_dir, images[j])
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            axes[i, j].imshow(img)
            axes[i, j].set_title(f'{class_name.capitalize()} - {images[j]}')
            axes[i, j].axis('off')
    
    plt.tight_layout()
    plt.show()

show_sample_images('../data', class_names)

## 2. Data Preprocessing

In [ ]:
# Clean up corrupted images
preprocessor.cleanup_dataset()

# Create data generators
train_generator, validation_generator, test_generator = preprocessor.create_data_generators(batch_size=32)

print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Class indices: {train_generator.class_indices}")

In [ ]:
# Visualize augmented images
def show_augmented_images(generator, num_images=8):
    batch_images, batch_labels = next(generator)
    
    plt.figure(figsize=(15, 8))
    for i in range(min(num_images, len(batch_images))):
        plt.subplot(2, 4, i + 1)
        plt.imshow(batch_images[i])
        
        # Get class name from label
        class_idx = np.argmax(batch_labels[i])
        class_name = list(train_generator.class_indices.keys())[class_idx]
        plt.title(f'Class: {class_name}')
        plt.axis('off')
    
    plt.suptitle('Augmented Training Images')
    plt.tight_layout()
    plt.show()

show_augmented_images(train_generator)

## 3. Model Architecture

In [ ]:
# Define custom CNN model
def create_custom_cnn(input_shape=(224, 224, 3), num_classes=3):
    model = Sequential([
        # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        
        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        
        # Third Convolutional Block
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        
        # Fourth Convolutional Block
        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        
        # Flatten and Dense Layers
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# Create and compile the model
model = create_custom_cnn()
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model architecture
model.summary()

In [ ]:
# Transfer Learning with VGG16
def create_transfer_learning_model(base_model_name='VGG16', input_shape=(224, 224, 3), num_classes=3):
    # Load pre-trained base model
    if base_model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'InceptionV3':
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # Freeze base model layers
    base_model.trainable = False
    
    # Add custom classification head
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# Create transfer learning model
transfer_model = create_transfer_learning_model('VGG16')
transfer_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Transfer Learning Model Summary:")
transfer_model.summary()

## 4. Training

In [ ]:
# Define callbacks
callbacks = [
    ModelCheckpoint(
        '../models/best_model.tf',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        mode='max',
        verbose=1,
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

# Train the custom CNN model
print("Training Custom CNN Model...")
history_cnn = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Train transfer learning model
print("Training Transfer Learning Model...")
callbacks_transfer = [
    ModelCheckpoint(
        '../models/best_transfer_model.tf',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        mode='max',
        verbose=1,
        restore_best_weights=True
    )
]

history_transfer = transfer_model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    callbacks=callbacks_transfer,
    verbose=1
)

## 5. Evaluation

In [ ]:
# Plot training history
def plot_training_history(history, title="Model Training History"):
    plt.figure(figsize=(15, 5))
    
    # Plot accuracy
    plt.subplot(1, 3, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    
    # Plot loss
    plt.subplot(1, 3, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    # Plot learning rate (if available)
    plt.subplot(1, 3, 3)
    if 'lr' in history.history:
        plt.plot(history.history['lr'], label='Learning Rate')
        plt.title('Learning Rate')
        plt.xlabel('Epoch')
        plt.ylabel('Learning Rate')
        plt.legend()
        plt.grid(True)
    else:
        plt.text(0.5, 0.5, 'Learning Rate\nNot Recorded', 
                ha='center', va='center', transform=plt.gca().transAxes)
    
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

# Plot training histories
plot_training_history(history_cnn, "Custom CNN Training History")
plot_training_history(history_transfer, "Transfer Learning Training History")

In [ ]:
# Evaluate models on validation data
def evaluate_model(model, validation_generator, model_name):
    print(f"\n{model_name} Evaluation:")
    print("=" * 50)
    
    # Reset generator
    validation_generator.reset()
    
    # Get predictions
    predictions = model.predict(validation_generator, verbose=1)
    predicted_classes = np.argmax(predictions, axis=1)
    
    # Get true labels
    true_classes = validation_generator.classes[:len(predicted_classes)]
    
    # Calculate accuracy
    accuracy = accuracy_score(true_classes, predicted_classes)
    print(f"Validation Accuracy: {accuracy:.4f}")
    
    # Classification report
    class_names = list(validation_generator.class_indices.keys())
    report = classification_report(true_classes, predicted_classes, 
                                 target_names=class_names, output_dict=True)
    
    print("\nClassification Report:")
    print(classification_report(true_classes, predicted_classes, target_names=class_names))
    
    # Confusion matrix
    cm = confusion_matrix(true_classes, predicted_classes)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'{model_name} - Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    
    return report, accuracy

# Evaluate both models
cnn_report, cnn_accuracy = evaluate_model(model, validation_generator, "Custom CNN")
transfer_report, transfer_accuracy = evaluate_model(transfer_model, validation_generator, "Transfer Learning")

In [ ]:
# Compare model performances
def compare_models(cnn_report, transfer_report, cnn_acc, transfer_acc):
    metrics = ['precision', 'recall', 'f1-score']
    classes = ['rose', 'tulip', 'sunflower']
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    for i, metric in enumerate(metrics):
        cnn_scores = [cnn_report[cls][metric] for cls in classes]
        transfer_scores = [transfer_report[cls][metric] for cls in classes]
        
        x = np.arange(len(classes))
        width = 0.35
        
        axes[i].bar(x - width/2, cnn_scores, width, label='Custom CNN', alpha=0.8)
        axes[i].bar(x + width/2, transfer_scores, width, label='Transfer Learning', alpha=0.8)
        
        axes[i].set_xlabel('Classes')
        axes[i].set_ylabel(metric.capitalize())
        axes[i].set_title(f'{metric.capitalize()} Comparison')
        axes[i].set_xticks(x)
        axes[i].set_xticklabels(classes)
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print("\nModel Comparison Summary:")
    print("=" * 40)
    print(f"Custom CNN Accuracy: {cnn_acc:.4f}")
    print(f"Transfer Learning Accuracy: {transfer_acc:.4f}")
    print(f"Best Model: {'Transfer Learning' if transfer_acc > cnn_acc else 'Custom CNN'}")

compare_models(cnn_report, transfer_report, cnn_accuracy, transfer_accuracy)

## 6. Visualization

In [ ]:
# Visualize predictions on sample images
def visualize_predictions(model, validation_generator, num_samples=9):
    validation_generator.reset()
    batch_images, batch_labels = next(validation_generator)
    
    predictions = model.predict(batch_images[:num_samples])
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(batch_labels[:num_samples], axis=1)
    
    class_names = list(validation_generator.class_indices.keys())
    
    plt.figure(figsize=(15, 15))
    for i in range(num_samples):
        plt.subplot(3, 3, i + 1)
        plt.imshow(batch_images[i])
        
        true_class = class_names[true_classes[i]]
        pred_class = class_names[predicted_classes[i]]
        confidence = predictions[i][predicted_classes[i]] * 100
        
        color = 'green' if true_classes[i] == predicted_classes[i] else 'red'
        
        plt.title(f'True: {true_class}\nPred: {pred_class}\nConf: {confidence:.1f}%', 
                 color=color, fontsize=12)
        plt.axis('off')
    
    plt.suptitle('Model Predictions on Validation Images', fontsize=16)
    plt.tight_layout()
    plt.show()

# Visualize predictions for both models
print("Custom CNN Predictions:")
visualize_predictions(model, validation_generator)

print("\nTransfer Learning Predictions:")
visualize_predictions(transfer_model, validation_generator)

In [ ]:
# Feature visualization - Conv layer activations
def visualize_conv_layers(model, sample_image, layer_names=None):
    if layer_names is None:
        # Get first few conv layers
        layer_names = [layer.name for layer in model.layers if 'conv' in layer.name.lower()][:4]
    
    # Create models that output activations
    activation_models = [Model(inputs=model.input, outputs=model.get_layer(name).output) 
                        for name in layer_names]
    
    # Get activations
    activations = [activation_model.predict(np.expand_dims(sample_image, axis=0)) 
                  for activation_model in activation_models]
    
    # Plot activations
    fig, axes = plt.subplots(len(layer_names), 8, figsize=(20, len(layer_names) * 3))
    
    for i, (layer_name, activation) in enumerate(zip(layer_names, activations)):
        # Show first 8 feature maps
        for j in range(min(8, activation.shape[-1])):
            if len(layer_names) == 1:
                ax = axes[j]
            else:
                ax = axes[i, j]
            
            ax.imshow(activation[0, :, :, j], cmap='viridis')
            ax.set_title(f'{layer_name}\nFeature {j+1}')
            ax.axis('off')
    
    plt.tight_layout()
    plt.show()

# Get a sample image
validation_generator.reset()
sample_batch, _ = next(validation_generator)
sample_image = sample_batch[0]

print("Conv Layer Activations for Custom CNN:")
visualize_conv_layers(model, sample_image)

## 7. Model Export

In [ ]:
# Save the best performing model
best_model = transfer_model if transfer_accuracy > cnn_accuracy else model
best_model_name = "transfer_learning" if transfer_accuracy > cnn_accuracy else "custom_cnn"

# Save in TensorFlow format
best_model.save(f'../models/best_flower_model_{best_model_name}.tf')

# Save model metadata
metadata = {
    'model_type': best_model_name,
    'accuracy': float(max(transfer_accuracy, cnn_accuracy)),
    'precision': float(transfer_report['macro avg']['precision'] if transfer_accuracy > cnn_accuracy 
                     else cnn_report['macro avg']['precision']),
    'recall': float(transfer_report['macro avg']['recall'] if transfer_accuracy > cnn_accuracy 
                   else cnn_report['macro avg']['recall']),
    'f1_score': float(transfer_report['macro avg']['f1-score'] if transfer_accuracy > cnn_accuracy 
                     else cnn_report['macro avg']['f1-score']),
    'class_names': ['rose', 'tulip', 'sunflower'],
    'input_shape': [224, 224, 3],
    'training_date': pd.Timestamp.now().isoformat()
}

import json
with open(f'../models/model_metadata_{best_model_name}.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"Best model saved: {best_model_name}")
print(f"Model accuracy: {metadata['accuracy']:.4f}")
print(f"Metadata saved to: model_metadata_{best_model_name}.json")

In [ ]:
# Test the saved model
print("Testing saved model...")

# Load the saved model
loaded_model = tf.keras.models.load_model(f'../models/best_flower_model_{best_model_name}.tf')

# Test prediction
test_image = sample_batch[0:1]
prediction = loaded_model.predict(test_image)
predicted_class = np.argmax(prediction[0])
confidence = prediction[0][predicted_class] * 100

print(f"Predicted class: {['rose', 'tulip', 'sunflower'][predicted_class]}")
print(f"Confidence: {confidence:.2f}%")
print("Model loaded and tested successfully!")

## Summary

This notebook has demonstrated a complete machine learning workflow for flower classification:

1. **Data Exploration**: Analyzed dataset structure and class distribution
2. **Data Preprocessing**: Implemented data cleaning, augmentation, and generators
3. **Model Architecture**: Created both custom CNN and transfer learning models
4. **Training**: Trained models with proper callbacks and monitoring
5. **Evaluation**: Comprehensive evaluation with metrics and visualizations
6. **Visualization**: Feature maps and prediction visualizations
7. **Model Export**: Saved the best performing model for production use

The trained model is now ready for deployment in the Flask API server!